# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chinde,26.8418,88.0763,22.58,21,60,3.36,NP,1713707165
1,1,adamstown,-25.0660,-130.1015,22.47,75,94,11.22,PN,1713707050
2,2,loen,7.7601,168.2307,27.82,82,100,8.62,MH,1713707166
3,3,hadibu,12.6500,54.0333,29.08,69,67,1.86,YE,1713707167
4,4,bilibino,68.0546,166.4372,-17.47,95,100,0.96,RU,1713707167


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

plot = city_data_df.hvplot.points('Lng', 'Lat', size='Humidity', c='City', cmap='accent', frame_height=400, 
geo=True, tiles=True, colorbar=True, alpha=0.6)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) 
                                & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
#narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,lazaro cardenas,17.9583,-102.2000,24.03,68,0,0.94,MX,1713707198
101,101,kailua-kona,19.6406,-155.9956,22.98,81,0,1.54,US,1713707214
117,117,arraial do cabo,-22.9661,-42.0278,26.98,54,0,4.12,BR,1713707221
119,119,la mira,18.0350,-102.3266,24.23,58,0,0.55,MX,1713707223
151,151,enterprise,36.0252,-115.2419,22.62,29,0,2.06,US,1713707237
175,175,schofield barracks,21.4936,-158.0615,22.18,83,0,1.54,US,1713707247
189,189,lata,40.1629,-8.3327,26.38,38,0,2.63,PT,1713707253
211,211,jardin america,-27.0435,-55.2270,24.21,53,0,2.32,AR,1713707264
216,216,peniche,39.3558,-9.3811,22.38,65,0,3.20,PT,1713707266
250,250,changji,44.0167,87.3167,22.44,21,0,2.00,CN,1713707281


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].reset_index()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,59,lazaro cardenas,MX,17.9583,-102.2000,68,
1,101,kailua-kona,US,19.6406,-155.9956,81,
2,117,arraial do cabo,BR,-22.9661,-42.0278,54,
3,119,la mira,MX,18.0350,-102.3266,58,
4,151,enterprise,US,36.0252,-115.2419,29,
5,175,schofield barracks,US,21.4936,-158.0615,83,
6,189,lata,PT,40.1629,-8.3327,38,
7,211,jardin america,AR,-27.0435,-55.2270,53,
8,216,peniche,PT,39.3558,-9.3811,65,
9,250,changji,CN,44.0167,87.3167,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "circle:" + str(longitude) + "," + str(latitude) + "," + str(radius)
    params["bias"] = "proximity:" + str(longitude) + "," + str(latitude)
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    url = base_url + "?categories=" + params["categories"] + "&filter=" + params["filter"] + "&bias=" + params["bias"] + "&limit=" + str(params["limit"]) + "&apiKey=" + geoapify_key

    # Make and API request using the params dictionaty
    name_address = requests.get(url)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
kailua-kona - nearest hotel: Kona Seaside Hotel
arraial do cabo - nearest hotel: No hotel found
la mira - nearest hotel: Hotel La Loma
enterprise - nearest hotel: Highland Inn
schofield barracks - nearest hotel: The Inn At Schofield
lata - nearest hotel: Residencial Botânico
jardin america - nearest hotel: Residencial Brasilia
peniche - nearest hotel: Residencial Maciel
changji - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
imbituba - nearest hotel: Silvestre Praia Hotel
dassun - nearest hotel: Hotel Carig
las vegas - nearest hotel: Downtown Grand
san juan colorado - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
maua - nearest hotel: Go Inn Santo André
aral - nearest hotel: No hotel found


,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,59,lazaro cardenas,MX,17.9583,-102.2000,68,Hotel Sol del Pacífico
1,101,kailua-kona,US,19.6406,-155.9956,81,Kona Seaside Hotel
2,117,arraial do cabo,BR,-22.9661,-42.0278,54,No hotel found
3,119,la mira,MX,18.0350,-102.3266,58,Hotel La Loma
4,151,enterprise,US,36.0252,-115.2419,29,Highland Inn
5,175,schofield barracks,US,21.4936,-158.0615,83,The Inn At Schofield
6,189,lata,PT,40.1629,-8.3327,38,Residencial Botânico
7,211,jardin america,AR,-27.0435,-55.2270,53,Residencial Brasilia
8,216,peniche,PT,39.3558,-9.3811,65,Residencial Maciel
9,250,changji,CN,44.0167,87.3167,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points('Lng', 'Lat', size='Humidity', c='Hotel Name', cmap='accent', hover_cols=['Country'], 
geo=True, tiles=True, colorbar=True, alpha=0.6)

# Display the map
hotels_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,Country)